In [2]:
from __future__ import division
from __future__ import print_function

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.python.framework import ops

import matplotlib
from matplotlib import pyplot as plt

matplotlib.style.use('ggplot')
%matplotlib inline

# Things to look for before marching to build Deep Networks
--------

* **Using Scope Variables**: We can create deep networks by writing individual code for each layers. In such a case if we want to go deeper say 10 layers deep, writing the same code would be redundant. Hence, we use scope to make the process more dynamic
* **Optimization**: Gradient Descent optimizer is a good optimizer, But why not use something with momentum and adaptive learning rate. Such optimizers are shown to be more powerfull in many applications. Example: Try *RMSPROP (when doing Batch learning)* and *ADAM optimizer (for general cases)* 

In [ ]:
# Here we create a re-usable funtion to add linear networks layers
def linearActivation(numInputs, numOutputs):
    with tf.VariableScope("linear"):
        w = tf.get_variable(dtype='float32',
                           shape=[numInputs, numOutputs],
                           initializer=tf.random_normal_initializer(mean=0.0, stddev=0.1),
                           name='weight')
        b = tf.get_variable(dtype='float32',
                           shape=[numOuptuts],
                           initializer=tf.constant_initializer(),
                           name='bias')
        
        y_pred = tf.matmul(x,w) + b
        
        